In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from radio_data_utility import RadioV2Dataset as dataset
import argparse
import os
import json
import pickle

from torchvision import transforms

import torch
import numpy as np

from torch.utils.data import DataLoader
import torch.nn as nn

from tqdm import tqdm

import time
import random

In [3]:
with open('./generated_radio_datasets_split/raven_organsmnist_v2_train.pkl', 'rb') as f:
    raven_dataset_v2_organ_train = pickle.load(f)

with open('./generated_radio_datasets_split/raven_organsmnist_v2_val.pkl', 'rb') as f:
    raven_dataset_v2_organ_val = pickle.load(f)

with open('./generated_radio_datasets_split/raven_organsmnist_v2_test.pkl', 'rb') as f:
    raven_dataset_v2_organ_test = pickle.load(f)

In [4]:
# from drnet import Solver, train_epoch, validate, count_parameters
from mrnet import MRNet, train_epoch, validate, count_parameters, contrast_loss

In [5]:
RANDOM_SEED = 42

# Set seeds for reproducibility
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

In [6]:
IMG_SIZE = 80

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(IMG_SIZE),  # Resize to the specified size
    transforms.ToTensor(),
])

eval_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
])

In [7]:
print("Total samples in training set:", len(raven_dataset_v2_organ_train))
print("Total samples in validation set:", len(raven_dataset_v2_organ_val))
print("Total samples in test set:", len(raven_dataset_v2_organ_test))

Total samples in training set: 4400
Total samples in validation set: 550
Total samples in test set: 550


In [8]:
train_dataset = dataset(
    raven_dataset_v2_organ_train, # Use the split training data
    mode="train",
    transform_train=train_transform,
    transform_eval=eval_transform, # transform_eval is not used by train mode but good to pass
)

val_dataset = dataset(
    raven_dataset_v2_organ_val, # Use the split validation data
    mode="val",    # Set mode to "val" or "test"
    transform_train=train_transform, # Not used by val mode
    transform_eval=eval_transform,
)

test_dataset = dataset(
    raven_dataset_v2_organ_test, # Use the split test data
    mode="val",    # Set mode to "val" or "test"
    transform_train=train_transform, # Not used by test mode
    transform_eval=eval_transform,
)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import time
import json
from sklearn.model_selection import StratifiedKFold, train_test_split
# ==============================================================================
# 1. SETUP - Constants and Parameters
# ==============================================================================
N_SPLITS = 3 # Number of folds for cross-validation
PATIENCE = 10 # Early stopping patience
VAL_SPLIT_SIZE = 0.20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 100
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
BETA1 = 0.9
BETA2 = 0.999
EPSILON = 1e-8
WEIGHT_DECAY = 1e-6

In [10]:
from torch.utils.data import TensorDataset, ConcatDataset, Subset, DataLoader

dev_dataset = ConcatDataset([train_dataset, val_dataset])
dev_dataset_labels = np.concatenate([train_dataset.labels, val_dataset.labels])

print(f"Total data for Development (Train+Val): {len(dev_dataset)} samples.")
print(f"Test Set size: {len(test_dataset)} samples. (Will not be used until the very end)")

Total data for Development (Train+Val): 4950 samples.
Test Set size: 550 samples. (Will not be used until the very end)


# RADIO-2 STL

In [11]:
model_folder_name = "RADIO2_MRNet"
from tensorboardX import SummaryWriter
import time

In [12]:
import time

print(f"\n{'='*25} PERFORMING NESTED CROSS-VALIDATION {'='*25}")

outer_kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
fold_test_results = []

for fold, (train_outer_idx, test_outer_idx) in enumerate(outer_kfold.split(np.zeros(len(dev_dataset)), dev_dataset_labels)):
    print(f"\n--- Outer Fold {fold+1}/{N_SPLITS} ---")
    writer = SummaryWriter(log_dir=f'runs/{model_folder_name}/fold_{fold+1}')

    # Create inner train/val split from the outer training set
    train_outer_labels = dev_dataset_labels[train_outer_idx]
    train_inner_idx, val_inner_idx = train_test_split(
        train_outer_idx, test_size=VAL_SPLIT_SIZE, shuffle=True, stratify=train_outer_labels, random_state=RANDOM_SEED
    )

    # Create subsets from the DEVELOPMENT dataset
    train_inner_subset = Subset(dev_dataset, train_inner_idx)
    val_inner_subset = Subset(dev_dataset, val_inner_idx)
    test_outer_subset = Subset(dev_dataset, test_outer_idx) # This is the "test set" for this fold

    train_loader = DataLoader(train_inner_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6, pin_memory=True)
    val_loader = DataLoader(val_inner_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)
    test_loader = DataLoader(test_outer_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)

    model = MRNet(
        use_meta=False,
        multihead=True,
    ).to(DEVICE)
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LEARNING_RATE,
        betas=(BETA1, BETA2),
        eps=EPSILON,
        weight_decay=WEIGHT_DECAY
    )
    criterion = lambda x, y, reduction='mean': contrast_loss(x, y, 'mean', False)
    scheduler = None
    scaler = None

    best_model_path = f"./saved_models/{model_folder_name}/best_model_fold_{fold+1}.pth"
    os.makedirs(f"./saved_models/{model_folder_name}", exist_ok=True)
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    print(f"Training on {len(train_inner_subset)}, Validating on {len(val_inner_subset)}...")
    for epoch in range(1, EPOCHS + 1):
        print(f"\nEpoch {epoch}/{EPOCHS} for Fold {fold+1}")
        train_start_time = time.time()
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
        train_end_time = time.time()
        train_time = train_end_time - train_start_time

        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Time: {train_time:.2f}s")

        writer.add_scalar('CV/Loss/train', train_loss, epoch)
        writer.add_scalar('CV/Accuracy/train', train_acc, epoch)
        writer.add_scalar('CV/Loss/val', val_loss, epoch)
        writer.add_scalar('CV/Accuracy/val', val_acc, epoch)
        writer.add_scalar('CV/Time/train', train_time, epoch)

        memory_allocated = torch.cuda.memory_allocated(DEVICE) if torch.cuda.is_available() else 0
        memory_reserved = torch.cuda.memory_reserved(DEVICE) if torch.cuda.is_available() else 0

        writer.add_scalar('CV/Memory/allocated', memory_allocated, epoch)
        writer.add_scalar('CV/Memory/reserved', memory_reserved, epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  New best model for fold {fold+1} at epoch {epoch}: Val Acc: {val_acc:.4f}")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    # Evaluate the best model for this fold on the outer test set
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    model.load_state_dict(torch.load(best_model_path))
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance on Outer Test Set for fold {fold+1}: {final_fold_test_acc:.4f}")
    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('CV/Accuracy/test', final_fold_test_acc, epoch)
    writer.close()

mean_cv_acc = np.mean(fold_test_results)
std_cv_acc = np.std(fold_test_results)
print(f"\n--- NESTED CROSS VALIDATION COMPLETE ---")
print(f"Cross-validation accuracies on outer folds: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Mean CV Accuracy: {mean_cv_acc:.4f} ± {std_cv_acc:.4f}")


========================= PERFORMING NESTED CROSS-VALIDATION =========================

--- Outer Fold 1/3 ---
LEVELS: 111
Training on 2640, Validating on 660...

Epoch 1/100 for Fold 1


Training: 100%|██████████| 21/21 [00:06<00:00,  3.00it/s]


Epoch Loss: 0.4812, Accuracy: 0.3280


Validating: 100%|██████████| 6/6 [00:01<00:00,  5.02it/s]


  Train Loss: 0.4812, Train Acc: 0.3280, Val Loss: 1.0560, Val Acc: 0.1818, Time: 6.99s
  New best model for fold 1 at epoch 1: Val Acc: 0.1818

Epoch 2/100 for Fold 1


...

Epoch Loss: 0.0116, Accuracy: 0.9996


Validating: 100%|██████████| 6/6 [00:01<00:00,  5.15it/s]


  Train Loss: 0.0116, Train Acc: 0.9996, Val Loss: 0.2986, Val Acc: 0.7864, Time: 5.67s
  Early stopping at epoch 47.
Loading best model for fold 3 (achieved 0.8167 on inner val set).


Validating: 100%|██████████| 13/13 [00:01<00:00,  6.80it/s]

Performance on Outer Test Set for fold 3: 0.7867

--- NESTED CROSS VALIDATION COMPLETE ---
Cross-validation accuracies on outer folds: ['0.7952', '0.8030', '0.7867']
Mean CV Accuracy: 0.7949 ± 0.0067


In [14]:
# ==============================================================================
# FINAL SUMMARY OF ALL RESULTS
# ==============================================================================

print(f"\n\n{'='*30} OVERALL EXPERIMENT SUMMARY {'='*30}")
print(f"1. Nested Cross Validation Performance: {mean_cv_acc:.4f} ± {std_cv_acc:.4f}")
# print(f"2. Final Deployable Model Performance (on test set): {final_benchmark_score:.4f}")



============================== OVERALL EXPERIMENT SUMMARY ==============================
1. Nested Cross Validation Performance: 0.7949 ± 0.0067


# RADIO-1 to RADIO-2 TL

In [15]:
from tensorboardX import SummaryWriter
import time

transfer_model_folder_name = "./saved_models/RADIO1_MRNet"
model_folder_name = "RADIO2_MRNet_TRANSFER"

print(f"\n{'='*25} PERFORMING NESTED CROSS-VALIDATION TRANSFER {'='*25}")

outer_kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
fold_test_results = []

for fold, (train_outer_idx, test_outer_idx) in enumerate(outer_kfold.split(np.zeros(len(dev_dataset)), dev_dataset_labels)):
    print(f"\n--- Outer Fold {fold+1}/{N_SPLITS} ---")
    writer = SummaryWriter(log_dir=f'runs/{model_folder_name}/fold_{fold+1}')

    # Create inner train/val split from the outer training set
    train_outer_labels = dev_dataset_labels[train_outer_idx]
    train_inner_idx, val_inner_idx = train_test_split(
        train_outer_idx, test_size=VAL_SPLIT_SIZE, shuffle=True, stratify=train_outer_labels, random_state=RANDOM_SEED
    )

    # Create subsets from the DEVELOPMENT dataset
    train_inner_subset = Subset(dev_dataset, train_inner_idx)
    val_inner_subset = Subset(dev_dataset, val_inner_idx)
    test_outer_subset = Subset(dev_dataset, test_outer_idx) # This is the "test set" for this fold

    train_loader = DataLoader(train_inner_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6, pin_memory=True)
    val_loader = DataLoader(val_inner_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)
    test_loader = DataLoader(test_outer_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)


    model = MRNet(
        use_meta=False,
        multihead=True,
    ).to(DEVICE)

    model.load_state_dict(torch.load(f"{transfer_model_folder_name}/deployable_model.pth"))  # Load the pre-trained model

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LEARNING_RATE,
        betas=(BETA1, BETA2),
        eps=EPSILON,
        weight_decay=WEIGHT_DECAY
    )
    criterion = lambda x, y, reduction='mean': contrast_loss(x, y, 'mean', False)
    scheduler = None
    scaler = None

    best_model_path = f"./saved_models/{model_folder_name}/best_model_fold_{fold+1}.pth"
    os.makedirs(f"./saved_models/{model_folder_name}", exist_ok=True)
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    print(f"Training on {len(train_inner_subset)}, Validating on {len(val_inner_subset)}...")
    for epoch in range(1, EPOCHS + 1):
        print(f"\nEpoch {epoch}/{EPOCHS} for Fold {fold+1}")
        train_start_time = time.time()
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
        train_end_time = time.time()
        train_time = train_end_time - train_start_time

        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Time: {train_time:.2f}s")

        writer.add_scalar('CV/Loss/train', train_loss, epoch)
        writer.add_scalar('CV/Accuracy/train', train_acc, epoch)
        writer.add_scalar('CV/Loss/val', val_loss, epoch)
        writer.add_scalar('CV/Accuracy/val', val_acc, epoch)
        writer.add_scalar('CV/Time/train', train_time, epoch)

        memory_allocated = torch.cuda.memory_allocated(DEVICE) if torch.cuda.is_available() else 0
        memory_reserved = torch.cuda.memory_reserved(DEVICE) if torch.cuda.is_available() else 0

        writer.add_scalar('CV/Memory/allocated', memory_allocated, epoch)
        writer.add_scalar('CV/Memory/reserved', memory_reserved, epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  New best model for fold {fold+1} at epoch {epoch}: Val Acc: {val_acc:.4f}")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    # Evaluate the best model for this fold on the outer test set
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    model.load_state_dict(torch.load(best_model_path))
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance on Outer Test Set for fold {fold+1}: {final_fold_test_acc:.4f}")
    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('CV/Accuracy/test', final_fold_test_acc, epoch)
    writer.close()

mean_cv_acc = np.mean(fold_test_results)
std_cv_acc = np.std(fold_test_results)
print(f"\n--- NESTED CROSS VALIDATION TRANSFER COMPLETE ---")
print(f"Cross-validation accuracies on outer folds: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Mean CV Accuracy: {mean_cv_acc:.4f} ± {std_cv_acc:.4f}")


========================= PERFORMING NESTED CROSS-VALIDATION TRANSFER =========================

--- Outer Fold 1/3 ---
LEVELS: 111
Training on 2640, Validating on 660...

Epoch 1/100 for Fold 1


Training: 100%|██████████| 21/21 [00:05<00:00,  3.70it/s]


Epoch Loss: 0.1792, Accuracy: 0.7867


Validating: 100%|██████████| 6/6 [00:01<00:00,  5.15it/s]


  Train Loss: 0.1792, Train Acc: 0.7867, Val Loss: 0.1370, Val Acc: 0.8318, Time: 5.68s
  New best model for fold 1 at epoch 1: Val Acc: 0.8318

Epoch 2/100 for Fold 1


...

Epoch Loss: 0.0214, Accuracy: 0.9909


Validating: 100%|██████████| 6/6 [00:01<00:00,  5.06it/s]


  Train Loss: 0.0214, Train Acc: 0.9909, Val Loss: 0.1934, Val Acc: 0.8606, Time: 5.75s
  Early stopping at epoch 15.
Loading best model for fold 3 (achieved 0.8727 on inner val set).


Validating: 100%|██████████| 13/13 [00:01<00:00,  6.73it/s]

Performance on Outer Test Set for fold 3: 0.8509

--- NESTED CROSS VALIDATION TRANSFER COMPLETE ---
Cross-validation accuracies on outer folds: ['0.8376', '0.8642', '0.8509']
Mean CV Accuracy: 0.8509 ± 0.0109
